In [2]:
import matplotlib.pyplot as plt
import numpy as np
import warnings

from tools import *

def mask_in(coord_x,coord_y,rad,x_grid,y_grid=None):
    if y_grid ==None:
        y_grid = x_grid
    mask = np.ones(shape=(x_grid,y_grid))
    for i in range(len(mask)):
        for j in range(len(mask[0])):
            R = np.sqrt((j-coord_x)**2 +(i-coord_y)**2)
            if R<=rad:
                mask[i][j]=0
    return mask

def mask_out(coord_x,coord_y,rad,x_grid,y_grid=None):
    if y_grid ==None:
        y_grid = x_grid
    mask = np.ones(shape=(x_grid,y_grid))
    for i in range(len(mask)):
        for j in range(len(mask[0])):
            R = np.sqrt((j-coord_x)**2 +(i-coord_y)**2)
            if R>rad:
                mask[i][j]=0
    return mask

def create_mask(image,setting):
    numPix     = image.shape[0]
    if not hasattr(setting,"R_tot_mask"):
        R_tot_mask = numPix*1./2.
    else:
        R_tot_mask = setting.R_tot_mask
    # we mask all the objects
    mask = np.ones(shape=(numPix,numPix))
    for i in range(len(setting.x_mask)):
        mask*=mask_in(setting.x_mask[i] ,setting.y_mask[i],setting.r_mask[i],numPix) 
    
    # we mask everything further out than a certain radius
    if not hasattr(setting,"new_center_mask_out"):
        x_mask_out,y_mask_out = numPix/2,numPix/2
    else:
        x_mask_out,y_mask_out = setting.new_center_mask_out
        
    mask*=mask_out(x_mask_out,y_mask_out,rad=R_tot_mask,x_grid=numPix)
    
    # We mask eventual residuals in center (due to lens light modelling imperfections)
    for i in range(len(setting.x_mask_cen)):
        mask*=mask_in(coord_x=setting.x_mask_cen[i],\
                      coord_y=setting.y_mask_cen[i],\
                      rad=setting.rad_mask_cen[i],\
                      x_grid=numPix)
    return mask

In [3]:
from astropy.io import fits

def load_fits(image_path):
    #load the image and read it as numpy array
    with fits.open(image_path) as hdulist:
        image   = hdulist[0].data
    return image

def subtract_light(image,setting):
    # Input the image and the setting file
    # Output the image with subtracted lens light
    if setting.sub==True:
        if setting.lens_light_model_name is None:
            print("Warning: no lens light model to subtract. I assume the input image was already subtracted.\n")
            image_sub = image
        else:
            lens_light_model_path = setting.data_path+"/"+setting.lens_light_model_name
            image_model = load_fits(lens_light_model_path)
            #We substract from the original image
            image_sub = image - image_model
    else:
        return image
    return image_sub

def get_transf_matrix(image_path_or_setting,in_arcsec=True):
    if type(image_path_or_setting)==str:
        if ".fits" in image_path_or_setting:
            image_path = image_path_or_setting
    else:
        setting = get_setting_module(image_path_or_setting).setting()
        image_path = setting.data_path+"/"+setting.image_name

    with fits.open(image_path) as hdulist:
        hdr = hdulist[0].header
    CD1_1,CD1_2,CD2_1,CD2_2 = hdr["CD1_1"],hdr["CD1_2"],hdr["CD2_1"],hdr["CD2_2"]
    transform_pix2angle = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
    if in_arcsec:
        transform_pix2angle*=3600
    return transform_pix2angle


In [4]:
def plot_image(image,setting,savefig_path_name=None,err_image=False):
    cmap_string = 'gist_heat'
    cmap = plt.get_cmap(cmap_string)
    cmap.set_bad(color='k', alpha=1.)
    cmap.set_under('k')
    if not err_image:
        if hasattr(setting, 'v_min'):
            v_min = setting.v_min
            v_max = setting.v_max
        else:
            v_min = -4
            v_max = 1
    else:
        if hasattr(setting, 'e_v_min'):
            v_min = setting.e_v_min
            v_max = setting.e_v_max
        else:
            v_min = -1
            v_max = 0
    f, ax = plt.subplots(1, 1, figsize=(6, 6), sharex=False, sharey=False)
    ax.matshow(np.log10(image), origin='lower',  vmin=v_min, vmax=v_max, cmap=cmap, extent=[0, 1, 0, 1])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.autoscale(False)
    if savefig_path_name is None:
        return ax
    else:
        plt.savefig(savefig_path_name)

In [5]:
def crop_egdes(image,setting):
    # Crop image
    if not hasattr(setting, 'edge'):
        raise RuntimeError("Setting file has no edge")
    else:
        edge = getattr(setting, "edge")
    if hasattr(setting, 'dx'):
        dx = setting.dx
        dy = setting.dy
    else:
        dx = int(setting.center_x_pix)
        dy = int(setting.center_y_pix)

    x_min,x_max = dx - edge, dx + edge
    y_min,y_max = dy - edge, dy + edge
    
    image_cut = []
    for i in range(len(image)):
        im_line =[]
        if i>y_min and i<y_max:
            for j in range(len(image[i])):
                if j>x_min and j<x_max:
                    im_line.append(image[i][j])
            image_cut.append(im_line)
    image_cut = np.array(image_cut)
    return image_cut

In [ ]:
def psf_correction(psf,setting):
    corr_psf = psf
    if setting.pssf>1:
        dl_mean0 = np.mean(psf[0])
        dl_mean1 = np.mean(psf[:,0])
        corr_psf = np.ones((len(psf)+2,len(psf[0])+2))
        corr_psf[0]*=dl_mean0
        corr_psf[-1]*=dl_mean0
        corr_psf[:,0]*=dl_mean1
        corr_psf[:,-1]*=dl_mean1
        corr_psf[0][0]=dl_mean0
        corr_psf[0][-1]=dl_mean0
        corr_psf[-1][-1]=dl_mean1
        corr_psf[-1][0]=dl_mean1

        for i in range(1,len(corr_psf)-1):
            for j in range(1,len(corr_psf[0])-1):
                corr_psf[i][j]=psf[i-1][j-1]
    return corr_psf

In [ ]:
def get_numPix(setting_name):
    setting_module = get_setting_module(setting_name) 
    setting        = setting_module.setting()
    image_path = setting.data_path+setting.image_name
    image      = load_fits(image_path)
    numPix     = image.shape
    if numPix[0] == numPix[1]:
        return numPix[0]
    else: #pragma: no cover
        raise RuntimeError("Image must be squared, numPix should be indentical in both xy not",numPix)

In [ ]:

def get_pixscale(setting_or_tm,only_one=True):
    # check
    # https://danmoser.github.io/notes/gai_fits-imgs.html
    if type(setting_or_tm)!=type(np.array([])) or type(setting_or_tm)!=list:
        transform_pix2angle = get_transf_matrix(setting_or_tm,False)
    else:
        transform_pix2angle = setting_or_tm
    pix_scale1 = np.sqrt(transform_pix2angle[0][0]**2 + transform_pix2angle[1][0]**2)
    pix_scale2 = np.sqrt(transform_pix2angle[0][1]**2 + transform_pix2angle[1][1]**2)
    if only_one:
        if abs(pix_scale1-pix_scale2)>min([pix_scale1,pix_scale2])*.1/100.: # pragma no cover
            # if we are considering only one and the difference is higher then 0.1 % of the smaller pix_scale
            # we raise and error
            raise RuntimeError("Pixel scale significantly different for the x and y axis")
        return pix_scale1
    else: 
        return pix_scale1, pix_scale2
        
def get_rotangle(setting_or_tm,in_deg=True):
    # check
    # https://danmoser.github.io/notes/gai_fits-imgs.html
    # get rotation angle of FoR with respect to pixel coordinates 
    # hence to rotate it correctly we need to rotate the angle by the inverse of this
    if type(setting_or_tm)!=type(np.array([])) or type(setting_or_tm)!=list:
        transform_pix2angle = get_transf_matrix(setting_or_tm,False)
    else:
        transform_pix2angle = setting_or_tm
    rot_angle = np.arctan2(transform_pix2angle[1][0],transform_pix2angle[0][0])
    if not in_deg:
        warnings.warn("Return rotation angle in rad") 
        return rot_angle
    else:
        warnings.warn("Return rotation angle in degrees") 
        return rot_angle*180/np.pi